In [1]:
import sys, os, time, shutil, random
from pathlib import Path
if '_cwd' not in locals():
    _cwd = os.getcwd()
    os.chdir(Path(_cwd)/'..')

import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
from tabulate import tabulate
import matplotlib.patches as mpatches
from tqdm.notebook import tqdm
from scipy import misc, signal

from sklearn.neighbors import NearestNeighbors
import networkx as nx
import cv2
import random
import ctypes
from scipy.signal import convolve2d, correlate2d
import ipywidgets as widgets
from IPython.display import display, clear_output
from collections import Counter
import io

import scipy.sparse as sp
from scipy.optimize import linear_sum_assignment

from omnibelt import load_yaml, save_yaml, multi_index

from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.segmentation import *
from skimage.segmentation import watershed, expand_labels
from skimage.morphology import closing, square, dilation
from skimage.color import label2rgb
from PIL import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
root = Path('extra/wd/step1')

In [3]:
rgba = Image.open(root/'wd_color.png').convert('RGBA')
rgba = np.asarray(rgba)
rgb = rgba[...,:3]
lbls = np.array(Image.open(root/'regions.png'))
tileimg = np.array(Image.open(root/'tiles.png'))
lbls.shape, rgb.shape, tileimg.shape

((2232, 5988), (2232, 5988, 3), (2232, 5988))

In [4]:
def showx(im, cover=lbls, fgax=None):
    if fgax is None:
        fgax = plt.subplots(figsize=(18,11))
    fg, ax = fgax
    ax.cla()
    plt.sca(ax)
    try:
        plt.imshow(im.astype(int))
        if cover is not None:
            plt.imshow(cover%1000, alpha=0., zorder=10)
    except ValueError:
        plt.imshow(im)
    
    plt.axis('off')
    plt.subplots_adjust(0,0,1,1)
    return fgax
    
    

# def get_highlight(rgb, mask, opacity=0.2):
#     alpha = np.zeros_like(mask).astype(np.uint8)
    
#     img = rgb.astype(np.uint8)
    
#     alpha[mask!=0] = 255
#     alpha[mask==0] = int(255*opacity)
    
#     H,W,C = img.shape
    
#     if C == 4:
#         img[...,-1] = alpha
#     else:
#         img = np.concatenate([img, alpha.reshape(H,W,1)],-1)
#     return img
    
# def highlight(rgb, mask, opacity=0.2, cover=lbls):
#     img = get_highlight(rgb, mask, opacity=opacity)
#     return showx(img, cover=cover)
    
    
    
def get_highlight(mask, _rgb=rgb, opacity=0.2):
    alpha = np.zeros_like(mask).astype(np.uint8)
    
    img = _rgb.astype(np.uint8)
    
    alpha[mask!=0] = 255
    alpha[mask==0] = int(255*opacity)
    
    H,W,C = img.shape
    
    if C == 4:
        img[...,-1] = alpha
    else:
        img = np.concatenate([img, alpha.reshape(H,W,1)],-1)
    return img
    
def highlight(mask, _rgb=rgb, opacity=0.2, cover=lbls, fgax=None):
    img = get_highlight(mask, _rgb, opacity=opacity)
    return showx(img, cover=cover, fgax=fgax)
    


    
# showx(lbls)

In [5]:
graph = load_yaml(root/'graph.yaml')
nodeIDs = {node['ID']:node for node in graph.values()}
regs = {node['name']:node for node in graph.values()}

bgs = load_yaml(root/'bgs.yaml')
bgIDs = {node['ID']:node for node in bgs.values()}

len(graph), len(bgs)

(743, 38)

In [8]:
# acolors = set(map(tuple,rgb.reshape(-1,3).tolist()))
# acolors = list(acolors)
acolors = [(174, 208, 210), (255, 174, 201), (181, 230, 29), (0, 255, 255), (175, 171, 100), (172, 205, 207), (135, 207, 213), (255, 242, 0), (255, 0, 128), (67, 67, 68), (171, 203, 205), (128, 128, 0), (66, 68, 68), (239, 228, 176), (0, 72, 78), (81, 81, 0), (127, 127, 127), (120, 114, 88), (195, 195, 195), (34, 177, 76), (0, 128, 64), (66, 69, 69), (255, 255, 255), (175, 209, 211), (163, 73, 164), (255, 127, 39), (64, 128, 128), (171, 204, 206), (198, 79, 0), (71, 79, 80), (0, 0, 0), (64, 64, 64), (82, 37, 82), (65, 67, 68), (200, 191, 231), (0, 255, 128), (67, 69, 69), (64, 0, 128), (136, 0, 21), (255, 255, 254), (67, 68, 68), (64, 65, 65), (0, 162, 232), (69, 141, 147), (185, 122, 87), (191, 0, 120), (237, 28, 36), (65, 67, 67), (174, 94, 94), (64, 64, 60), (174, 207, 209), (255, 169, 132), (63, 72, 204), (171, 205, 206)]
len(acolors)

54

In [41]:
colornames = [None]*len(acolors)

In [68]:
def get_colormask(c):
    c = np.array(c).reshape(1,1,3)
    mask = np.abs(c-rgb).sum(-1) == 0
    return mask

In [44]:
fgax = None

todo = iter(tqdm([i for i, x in enumerate(colornames) if x is None]))
prompt = widgets.Label('Player name: ')
textbox = widgets.Text()
# response = widgets.Label()

def _update_display():
    try:
        idx = next(todo)
    except StopIteration:
        if fgax is not None:
            plt.clf(fgax[0].num)
        print('Done!')
    else:
        textbox.value = ''
        textbox.index = idx
        display(widgets.HBox([prompt, textbox]))
        global fgax
        fgax = highlight(expand_labels(get_colormask(acolors[idx]), 20), fgax=fgax)
        
        
def _on_submit_func(sender):
    idx = textbox.index
    colornames[idx] = textbox.value
    _update_display()
textbox.on_submit(_on_submit_func)

_update_display()

In [49]:
# colornames = ['', 'Spanish Empire', 'Italian Empire', 'Argentine Republic', 'Omani Empire', '', '', 'Qing Empire', 'Japanese Empire', '', '', '', '', '', '', '', 'German Empire', '', '', 'Brazilian Empire', 'Ethiopian Empire', '', '', '', 'Russian Empire', 'Dutch Empire', 'Qajar Empire', '', 'Chilean Republic', '', '', '', '', '', 'Siamese Kingdom', 'Sweden-Norway', '', '', 'Austria-Hungary', 'Portuguese Empire', '', '', 'French Empire', '', 'Zulu Kingdom', 'Gran Colombia', 'British Empire', '', 'Ottoman Empire', '', '', 'Mexican Republic', 'United States', '']

In [84]:
tohex = lambda color: '#' + ''.join(hex(c)[2:].zfill(2) for c in color)

In [85]:
players = {}
for name, color in tqdm(zip(colornames, acolors), total=len(acolors)):
    if len(name):
        players[name] = {'color': tohex(color), 
                         'army': [], 'fleet': [],
                         'territory':[nodeIDs[i]['name'] for i in set(lbls[get_colormask(color)].tolist()) if i in nodeIDs]}
len(players)

  0%|          | 0/54 [00:00<?, ?it/s]

24

In [87]:
# players

In [88]:
# save_yaml(players, root/'players.yaml');